In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# cwd = os.getcwd()
cwd = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
dir_name = "m2f2O025"  #"Beauge"
mu_coeff = int(dir_name[1])
eta_coeff = int(dir_name[3])


In [ ]:
def list_matching_files_with_numbers(dir_name):
    matching_files = []
    dpy_pattern = re.compile(r"^dpy\d+$")  # Matches "dpy<int>"
    salida_pattern = re.compile(r"^salida(\d+)\.out$")  # Captures number after "salida"

    if not os.path.isdir(dir_name):
        return []

    for dpy_dir in os.listdir(dir_name):
        dpy_path = os.path.join(dir_name, dpy_dir)
        if os.path.isdir(dpy_path) and dpy_pattern.match(dpy_dir):
            for file in os.listdir(dpy_path):
                match = salida_pattern.match(file)
                if match:
                    salida_number = int(match.group(1))  # Extract the number
                    matching_files.append((os.path.join(dpy_path, file), salida_number))

    return matching_files

def read_simu(file):
    return pd.read_csv(file,
                       header=None,
                       sep='\s+',
                       index_col=None,
                       names=["nsim", "t", "a", "e", "M", "w", "R", "mass", "dist", "spin", "ast_mass"],
                       usecols=range(11),
                      )

In [ ]:
files = list_matching_files_with_numbers(os.path.join(cwd, dir_name))

In [ ]:
salida = None
if os.path.isfile(os.path.join(cwd, dir_name, "salida2.out")):
    salida = read_simu(os.path.join(cwd, dir_name, "salida.out"))
else:
    simus = {}
    for file_num in files:
        simus[file_num[1]] = read_simu(file_num[0])

In [ ]:
G = 4.9823394e-10  # [km^3 kg^(-1) day^(-2)]
if salida is not None:
    mu = G * salida.ast_mass.iloc[0]
    spin = salida.spin.iloc[0]
elif len(simus) > 0:
    mu = G * simus[1].ast_mass.iloc[0]
    spin = simus[1].spin.iloc[0]
else:
    raise ValueError("No data found")

In [ ]:
def a_from_ratio(ratio):
    one_n2 = (ratio / spin)**2
    return (mu * one_n2)**(1/3.)

def ratio_from_a(a):
    one_n = np.sqrt(a**3 / mu)
    return spin * one_n

In [ ]:
plt.figure(dpi=150)

if salida is not None:
    plt.plot(salida.t/365.25, salida.a, "r.", lw=0.4, ms=0.4, label="salida")
else:
    for simu in simus.values():
        plt.plot(simu.t.iloc[:-1]/365.25, simu.a.iloc[:-1], "k-", lw=0.4)

plt.semilogx()

plt.xlabel("time [yrs]")
plt.ylabel("$a$ [km]")

plt.title("$\mu=10^{-%d}$   $\eta=10^{-%d}$"%(mu_coeff, eta_coeff))

# plt.yticks(np.linspace(1.5, 4, 6))

# ax2 = plt.gca().secondary_yaxis("right", functions=(a_from_ratio, ratio_from_a))
# ax2.set_ylabel("$a$ [km]")

plt.tick_params(axis="both", top=True, right=False, direction="inout", which="both")
plt.grid(alpha=0.4)

plt.ylim(130, 350)

plt.show()

In [ ]:
plt.figure(dpi=150)

for simu in simus.values():
    plt.plot(simu.t/365.25, simu.R, "k-", lw=0.4)

plt.semilogx()

plt.xlabel("time [yrs]")
plt.ylabel("$\Omega/n$")

plt.title("$\mu=10^{-%d}$   $\eta=10^{-%d}$"%(mu_coeff, eta_coeff))

plt.yticks(np.linspace(1.5, 4, 6))

# ax2 = plt.gca().secondary_yaxis("right", functions=(a_from_ratio, ratio_from_a))
# ax2.set_ylabel("$a$ [km]")

plt.tick_params(axis="both", top=True, right=False, direction="inout", which="both")
plt.grid(alpha=0.4)

plt.show()